In [90]:
# Code to create a model used to predict whether the individual patient has an allergy
# Use different notebook to load the model and return a prediction

import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import tensorflow
import keras

import numpy as np
np.random.seed(123)

In [91]:
import matplotlib
from keras.models import * 
from keras.layers import *
from keras.optimizers import RMSprop
import pandas as pd

# Import both datasets, change to local path when running
patients = pd.read_excel(r"C:\Users\me\OneDrive\Desktop\Senior Design\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level2_AI_Patient Traits")

allergies = pd.read_excel(r"C:\Users\me\OneDrive\Desktop\Senior Design\Allergy_SanFrancisco\PATIENTS_Nov_3_2023_V4_sfm-data.xlsx", sheet_name="Level1_Patient Allergens")

In [92]:
# Confirm that patient sheet imported correctly
# Comment below line before committing
# patients.head()

In [93]:
# Confirm allergy sheet imported correctly
# Comment line before committing
# allergies.head()

In [94]:
# Merge columns by ID if needed
patientAllergies = patients.merge(allergies, on = "SFM Id")
# Comment line before committing
# patientAllergies.head()

In [95]:
# Drop ID column for preprocessing - ID should have no effect on prediction
patientsNoId = patients.drop('SFM Id', axis = 1)

In [96]:
# Drop ID column for preprocessing - ID should have no effect on prediction
# TODO: complete preprocessing for allergies columns to have valid output
allergiesNoId = allergies.drop('SFM Id', axis = 1)

In [97]:
# TODO: Refine layers/sequences to actually make the model once we get it loading in correctly
# classes should be total number of outputs (1 for each ingredient)
n_classes = 164687

model = Sequential()

# Input layer
model.add(Dense(128, input_shape=(8,), activation='relu'))

# Hidden layers
model.add(Dense(2048, activation = "relu"))
model.add(Dense(256, activation = "relu"))

# Output layer
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [98]:
# Change columns from being object to an actual datatype
# Most likely required for all columns
patientsNoId['Gender'] = pd.Categorical(patientsNoId['Gender'])
patientsNoId['SkinTone'] = pd.Categorical(patientsNoId['SkinTone'])
patientsNoId['Country'] = pd.Categorical(patientsNoId['Country'])
patientsNoId.dtypes

Gender                      category
BirthYear                      int64
City                          object
State                         object
Country                     category
FitzpatrickSkinPhotoType      object
SkinTone                    category
SkinConditions                object
dtype: object

In [99]:
# Code to preprocess the SkinConditions column, tokenizing should be more recognizable to keras than regular text
## Delete if this idea does not work
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# Flatten entries into actual lists
patientsNoId['SkinConditions'] = patientsNoId['SkinConditions'].apply(lambda x: ' '.join(x))
# Tokenize the characters of the columns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patientsNoId['SkinConditions'])
# Create and pad sequences of the tokens
patientsNoId['SkinConditions'] = tokenizer.texts_to_sequences(patientsNoId['SkinConditions'])
patientsNoId['SkinConditions'] = pad_sequences(patientsNoId['SkinConditions'], padding='post')

In [100]:
# TODO: create a training/testing split for the data

In [ ]:
# TODO: Change hyperparameters of the model to train/test
history = model.fit(
   x_train, y_train, 
   batch_size = 128, 
   epochs = 20, 
   verbose = 1, 
   validation_data = (x_test, y_test)
)

In [ ]:
# Get the average prediction score across all epochs
score = model.evaluate(x_test, y_test, verbose = 0)
score #First number is loss, second is accuracy

In [ ]:
# TODO: Save Keras model as separate file